In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from kmodes.kprototypes import KPrototypes

from load_datasets import *
from duped_modules.gower_duped import gower_matrix as gower_matrix_duped

In [2]:
soybean_df, soybean_y, soybean_cat_cols, soybean_cont_cols = load_soybean_disease()
heart_df, heart_y, heart_cat_cols, heart_cont_cols = load_heart_disease()
breast_df, breast_y, breast_cat_cols, breast_cont_cols = load_breast_cancer()
banking_df, banking_y, banking_cat_cols, banking_cont_cols = load_bank_marketing()
census_df, census_y, census_cat_cols, census_cont_cols = load_census_income()
credit_df, credit_y, credit_cat_cols, credit_cont_cols = load_credit_approval()

In [3]:
census_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0.025996,7,-1.061979,9,1.136512,4,1,1,4,1,0.146932,-0.217127,-0.034087,39
1,0.828308,6,-1.007104,9,1.136512,2,4,0,4,1,-0.144804,-0.217127,-2.213032,39
2,-0.046942,4,0.246034,11,-0.419335,0,6,1,4,1,-0.144804,-0.217127,-0.034087,39
3,1.047121,4,0.426663,1,-1.197259,2,6,0,2,1,-0.144804,-0.217127,-0.034087,39
4,-0.776316,4,1.408530,9,1.136512,2,10,5,2,0,-0.144804,-0.217127,-0.034087,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.025996,4,0.243884,9,1.136512,0,10,1,4,0,-0.144804,-0.217127,-0.356894,39
48838,1.849433,0,1.247492,11,-0.419335,6,0,2,2,1,-0.144804,-0.217127,-0.034087,39
48839,-0.046942,4,1.754865,9,1.136512,2,10,0,4,1,-0.144804,-0.217127,0.772930,39
48840,0.390683,4,-1.001612,9,1.136512,0,1,3,1,1,0.587220,-0.217127,-0.034087,39
